In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('C:\\Users\\Mustapha\\Desktop\\ML-Project-Scratch\\data\\raw\\untouched_raw_original.csv')

In [3]:
pd.set_option('display.max_columns', None) # To display all columns
pd.set_option('display.max_rows', None) # To display all rows

In [4]:
df.shape

(884092, 39)

In [5]:
df.head(2)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,year,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
0,2012-03-31,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,0.943662,0.142857,0.043478,ATL,30002,2012,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,Atlanta-Sandy Springs-Alpharetta
1,2012-04-30,61870.0,245000.0,40.723982,130.528256,22.0,29.0,56.0,69.0,89.5,0.946642,0.090909,0.034483,ATL,30002,2012,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,202421.064584,Atlanta-Sandy Springs-Alpharetta


In [6]:
df['date'] = pd.to_datetime(df['date'])
print(df['date'].dt.year.value_counts())

date
2013    74712
2014    74712
2015    74712
2016    74712
2017    74712
2018    74712
2019    74712
2020    74712
2021    74712
2022    74712
2023    74712
2012    62260
Name: count, dtype: int64


##### Avoid Leakage

- train set (2012–2019) is “clean” historical training.

- eval set (2020–2021) is “unseen” during training, but still used for validation and hyperparameter tuning.

- holdout set (2022–2023) is completely untouched until the very end.

In [11]:
# Sort by date (important to avoid shuffling issues)
df = df.sort_values("date")

In [12]:
cutoff_date_eval = "2020-01-01" # validation set starts from Jan 1, 2020
cutoff_date_holdout = "2022-01-01" # holdout set starts from Jan 1, 2022

In [14]:
# train set before 2020
train_df = df[df['date'] < cutoff_date_eval]

# evaluation/validation set between 2020 and 2021
eval_df = df[(df['date'] >= cutoff_date_eval) & (df['date'] < cutoff_date_holdout)]

# holdout/test set from 2022 onwards
holdout_df = df[df['date'] >= cutoff_date_holdout]

print(f"Training set shape: {train_df.shape}")
print(f"Evaluation set shape: {eval_df.shape}")
print(f"Holdout set shape: {holdout_df.shape}")

Training set shape: (585244, 39)
Evaluation set shape: (149424, 39)
Holdout set shape: (149424, 39)


In [15]:
train_df.head(2)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,year,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
0,2012-03-31,46550.0,217450.0,31.813674,110.183666,14.0,23.0,44.0,64.0,59.5,0.943662,0.142857,0.043478,ATL,30002,2012,12.0,2.0,4.0,1.0,60.0,45.0,57.0,4.0,7.0,5811.0,36.3,33052.0,5811.0,2677.0,710.0,279500.0,3171.0,460.0,5408.0,5408.0,2492.0,200773.999557,Atlanta-Sandy Springs-Alpharetta
218396,2012-03-31,234650.0,265000.0,130.620985,154.023973,81.0,101.0,149.0,174.0,95.0,0.933685,0.086420,0.148515,DAL,76107,2012,0.0,0.0,7.0,0.0,50.0,2.0,96.0,1.0,3.0,25530.0,35.9,40406.0,25143.0,13685.0,782.0,179700.0,14117.0,988.0,24264.0,24264.0,12291.0,158643.875925,Dallas-Fort Worth-Arlington


In [16]:
eval_df.head(2)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,year,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
162684,2020-01-31,268000.0,275900.0,117.649979,121.116709,105.0,98.0,87.0,75.0,41.5,0.984043,0.152381,0.459184,CIN,45066,2020,10.0,0.0,1.0,0.0,14.0,16.0,11.0,0.0,3.0,25040.0,38.4,45191.0,24856.0,9042.0,1172.0,240000.0,13025.0,331.0,23976.0,23976.0,11917.0,288036.180506,Cincinnati
161974,2020-01-31,304000.0,225900.0,147.213412,147.840376,7.0,9.0,11.0,9.0,53.0,0.961889,0.142857,0.444444,CIN,45054,2020,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,2924.0,39.2,44591.0,2924.0,1159.0,815.0,252200.0,1829.0,18.0,2824.0,2824.0,1711.0,278698.567188,Cincinnati


In [17]:
holdout_df.head(2)

,date,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,city,zipcode,year,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,city_full
501520,2022-01-31,375000.0,300000.0,234.700521,238.066826,23.0,25.0,21.0,10.0,31.0,1.019467,0.608696,0.000000,NY,7850,2022,7.0,0.0,0.0,1.0,10.0,27.0,20.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,388344.519628,New York-Newark-Jersey City
19998,2022-01-31,469900.0,529000.0,159.911570,189.378465,29.0,31.0,21.0,14.0,24.0,1.004782,0.517241,0.322581,ATL,30290,2022,3.0,0.0,0.0,2.0,10.0,6.0,12.0,0.0,2.0,8876.0,43.8,48314.0,8876.0,3428.0,932.0,313500.0,4364.0,135.0,8576.0,8576.0,3676.0,444243.222415,Atlanta-Sandy Springs-Alpharetta


In [ ]:
train_df.to_csv('C:\\Users\\Mustapha\\Desktop\\ML-Project-Scratch\\data\\raw\\train.csv', index=False)
eval_df.to_csv('C:\\Users\\Mustapha\\Desktop\\ML-Project-Scratch\\data\\raw\\eval.csv', index=False)
holdout_df.to_csv('C:\\Users\\Mustapha\\Desktop\\ML-Project-Scratch\\data\\raw\\holdout.csv', index=False)